# llama.cpp Fine-Tuning Template





llama.cpp helps to enable LLM inference with minimal setup and state-of-the-art performance on a wide variety of hardware.This allows the use of models packaged as .gguf files, which run efficiently in CPU-only and mixed CPU/GPU environments

`I prepared this llama.cpp fine-tuning template for my use case, but you could change it to suit your requirements.`



To View My Account:

* [Hugging Face ](https://huggingface.co/santhoshmlops)

* [Git Hub](https://github.com/santhoshmlops)

To View Some other Fine Tuning Template:

* [Fine Tuning Template ](https://github.com/santhoshmlops/MyHF_LLM_FineTuning/tree/main/FineTuningTemplate)


To View My Model Fine Tuning  NoteBook:

* [MY HF LLM Fine-Tuning](https://github.com/santhoshmlops/MyHF_LLM_FineTuning)



## Setting Up on Google Colab
Google Colab provides a convenient, cloud-based environment with access to powerful GPUs like the `T4`. If you choose Colab for this tutorial, make sure to select a GPU runtime by going to `Runtime > Change runtime type > T4 GPU`. This ensures that your notebook has access to the necessary computational resources.

## Setting Up Hugging Face Authentication

On Google Colab, you can safely store your Hugging Face token by using Colab's "Secrets" feature. This can be done by clicking on the "Key" icon in the sidebar, selecting "`Secrets`", and adding a new secret with the name `HF_TOKEN` and your Hugging Face token as the value. This method ensures that your token remains secure and is not exposed in your notebook's code.

#Things to change in this template



* Model Name
 - Check the llama.cpp Github page to find the Supported models before you proceed:  [Git Hub Page](https://github.com/ggerganov/llama.cpp)
* Quantization Methods
* Hugging Face User Name

## Quantization Methods

The names of the quantization methods follow the naming convention: "q" + the number of bits + the variant used (detailed below). Here is a list of all the possible quant methods and their corresponding use cases, based on model cards made by [TheBloke](https://huggingface.co/TheBloke/):

* `q2_k`: Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.
* `q3_k_l`: Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_m`: Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_s`: Uses Q3_K for all tensors
* `q4_0`: Original quant method, 4-bit.
* `q4_1`: Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.
* `q4_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
* `q4_k_s`: Uses Q4_K for all tensors
* `q5_0`: Higher accuracy, higher resource usage and slower inference.
* `q5_1`: Even higher accuracy, resource usage and slower inference.
* `q5_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
* `q5_k_s`:  Uses Q5_K for all tensors
* `q6_k`: Uses Q8_K for all tensors
* `q8_0`: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

As a rule of thumb, **I recommend using Q5_K_M** as it preserves most of the model's performance. Alternatively, you can use Q4_K_M if you want to save some memory. In general, K_M versions are better than K_S versions. I cannot recommend Q2_K or Q3_* versions, as they drastically decrease model performance.

# Step 1 - Clone the llama.cpp Git Repository

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 20790, done.
remote: Counting objects: 100% (3936/3936), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 20790 (delta 3854), reused 3819 (delta 3786), pack-reused 16854
Receiving objects: 100% (20790/20790), 23.11 MiB | 12.96 MiB/s, done.
Resolving deltas: 100% (14663/14663), done.


# Step 2 - Install the Requirements

In [3]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include 
I NVCCFLAGS: -std=c++11 -O3 -use_fast_math --forward-

#Step 3 - Initialize the Model Name and its Method to Quantize

In [15]:
from huggingface_hub import snapshot_download
model_name = "santhoshmlops/Skai-gemma-2b-it-SFT"  # Change the model name as your wish. For eg -"microsoft/phi-1_5"
quantization_methods = ['q4_k_m']   # Change the model quantization methods type as your wish. For eg - ['q5_k_m']
hf_user_name = "santhoshmlops"   # Change the HF User Name. For eg - "santhoshmlops"
base_model = "./original_model/"
quantized_path = "./quantized_model/"
qtype = f"{quantized_path}{quantization_methods[0].upper()}.gguf"
original_model = quantized_path+'/FP16.gguf'
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/78.5M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

'/content/original_model'

#Step 4 - Make a Directory for Quantized model

In [7]:
!mkdir ./quantized_model/
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
gguf: Setting special token type bos to 2
gguf: Setting special token type eos to 1
gguf: Setting special token type unk to 3
gguf: Setting special token type pad to 0
gguf: Setting add_bos_token to True
gguf: Setting add_eos_token to False
gguf: Setting chat_template to {{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '
' + message['content'] | trim + '<end_of_turn>
' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model
'}}{% endif %}
Exporting model to

# Step 5 - Build the Quantized Model

`Note:` You can stop the run cell once you're okay with the user interacting with Bob the Assistant

In [8]:
import os
for m in quantization_methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

! ./llama.cpp/main -m {qtype} -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

error: unknown argument: --repeat-penalty

usage: ./llama.cpp/main [options]

options:
  -h, --help            show this help message and exit
  --version             show version and build info
  -i, --interactive     run in interactive mode
  --interactive-first   run in interactive mode and wait for input right away
  -ins, --instruct      run in instruction mode (use with Alpaca models)
  -cml, --chatml        run in chatml mode (use with ChatML-compatible models)
  --multiline-input     allows you to write or paste multiple lines without ending each in '\'
  -r PROMPT, --reverse-prompt PROMPT
                        halt generation at PROMPT, return control in interactive mode
                        (can be specified more than once for multiple prompts).
  --color               colorise output to distinguish prompt and user input from generations
  -s SEED, --seed SEED  RNG seed (default: -1, use random seed for < 0)
  -t N, --threads N     number of threads to use during generat

# Step 6 - Login to your Hugging Face Hub
`Note:`  If you have already set the HF_TOKEN secret key, you can skip this step

In [5]:
from huggingface_hub import notebook_login
notebook_login()

# Step 7 -  Initialize the Model path and Repository name

In [13]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file
model_path = qtype
user_name = hf_user_name
repo_name = model_name.split("/")[-1]+"-GGUF"
repo_path = model_name.split("/")[-1].lower()+"."+model_path.split("/")[-1]
repo_id = user_name+"/"+repo_name
repo_type = "model"
repo_url = create_repo(repo_name, private=False)

# Step 8 - Push the Quantized model to Hub

In [14]:
api = HfApi()
api.upload_file(
    path_or_fileobj = model_path,
    path_in_repo = repo_path,
    repo_id = repo_id,
    repo_type = repo_type,
)

Q4_K_M.gguf:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/santhoshmlops/Skai-gemma-2b-it-SFT-GGUF/commit/ce94ba8c2d94a93728c4dfb25624be231451ebe9', commit_message='Upload Skai-gemma-2b-it-SFT.Q4_K_M.gguf with huggingface_hub', commit_description='', oid='ce94ba8c2d94a93728c4dfb25624be231451ebe9', pr_url=None, pr_revision=None, pr_num=None)